# 6. Korpuszépítés

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## Crawling és scraping

**crawling**: Weboldalak letöltése, keresőrobotokkal, jelemzően indexelés céljából. Gyakran rekuzív, azaz a kezdő oldalon található linkeket "követi", sít.

**scraping**: Hasonló, de hangsúlyosabb az adatkinyerés, az oldalak és egyéb dokumentumok feldolgozása. Nem feltétlenül rekurzív, gyakran csak egy oldal a tárgya.

**sitemap.xml**: Az oldalon elérhető lapokat sorolja fel, robotok által feldolgozható formában (XML).

**robots.txt**: Robotok számára tiltott ill. engedélyezett aloldalak részletezése. Figyelembevétele önkéntes, figyelembe nem vétele IP-cím letiltást eredményezhet. Sokszor tartalmazza sitemap.xml elérhetőségét is.

**Példák**

* helyesiras.mta.hu
    * http://helyesiras.mta.hu/robots.txt
    * http://helyesiras.mta.hu/sitemap.xml
* index.hu
    * https://index.hu/robots.txt
    * https://index.hu/sitemap/sitemapindex.xml

## Wget

Parancssoros letöltő program. 

Használat: `wget [OPTIONS] [URLS]`.

In [ ]:
%%bash

wget -O nytud.html www.nytud.hu/index.html

ls *.html

Pár hasznos kapcsoló:

* `-m` vagy `--mirror`: teljes (al)oldal (*site*) tükrözése könyvtárszerkezettel együtt, külső dolgok nélkül
* `-r` vagy `--recursive`: a megadott lapon található linkeket is letölti és az azokon találhatóakat is, sít.
    * `-l depth`: a rekuzív bejárás mélysége adható meg ezzel (default 5)
    * `-H`: kívülre mutató linkeket is letölti
* `-nd`: csak a fájlokat tölti le, nem hozza létre a könyvtárszerkezetet
* `-P path/to/dir/`: célkönyvtár megadása
* `-A acclist` ill. `-R rejlist`: elfogadandó és elutasítandó suffixek listája, pl. `-R "*.png,*.jpg"`
* `--accept-regex urlregex` ill. `--reject-regex urlregex`: elfogadandó ill. elutasítandó fájlnevek mintái

## BeautifulSoup

A BeautifulSoup egy HTML és XML fájlok elemzésére, átalakítására szolgáló python csomag.

In [ ]:
from bs4 import BeautifulSoup as BS

**Leveskészítés**

In [ ]:
# stringből

html = '''
<html>
    <head>
        <title>BeautifulSoup példa</title>
    </head>
    <body>
        <h1>A számítógépes nyelvészet örömei</h1>
        <p>Lórum ipse természetesen kukodik nyesedik a csevézmény számára.
        A kúra fakálásról felő matija szerint a szormányolt majháma 133
        tatott retleni helyismeszet perál a hatlan lenzők bucnitására.</p>
        <ul>
            <li>A számítógépes nyelvészet jó.</li>
            <li>A számítógépes nyelvészet szép.</li>
        </ul>
    </body>
</html>
'''

soup = BS(html, 'html.parser')
print(type(soup), '\n\n')
print(soup.prettify())

In [ ]:
# fájlból

soup = BS(open('nytud.html', encoding='latin2'), 'html.parser')
print(soup.prettify())

**Tag-ek**

In [ ]:
links = soup.find_all('a')
for link in links[:10]:
    print(link['href'])

**Szöveg**

In [ ]:
print(headers[0])
print()
print(headers[0].get_text())

## Tika

A Tika egy Java csomag, ami különböző fájlformátumok szöveges részeit tudja kezelni. Használható PDF, DOC, DOCX, HTML és más fájlokból szövegkinyerésre.

In [ ]:
%%bash

# wget xenia.sote.hu/ftp/mirrors/www.apache.org/tika/tika-app-1.17.jar
# cp /path/to/tika-app-1.17.jar .

# java -jar tika-app-1.17.jar --help

java -jar tika-app-1.17.jar -t proposal.pdf

## Feladatok

1. Töltsük le a http://clara.nytud.hu/~kk120/ oldal alatt található publikációkat html-ben.
    1. A lábjegyzeteket tartalmazó számozott html fájlok nem kellenek (pl. *acs3.html*).
2. BeautifulSoup segítségével az egyes html fájlokból nyerjük ki a bekezdéseik szövegét, mentsük ezeket txt-ként (azaz egy txt fájl tartalmazza egy html fájl összes bekezdését).
3. Töltsük le a http://clara.nytud.hu/~kk120/ oldal alatt található pdf fájlokat.
4. Írjunk python kódot, ami a Tika segítségével szöveges fájlokká alakítja ezeket.

**Segítség**

List comprehension: ha for ciklussal akarunk egy már meglévő listából egy másik listát készíteni így:

```python
l1 = [1, 2, 3]
l2 = []
for i in l1:
    l2.append(i**2)
# l2 = [1, 4, 9]
```

akkor lehet ezt is:

```python
l1 = [1, 2, 3]
l2 = [x**2 for x in l1]
# l2 = [1, 4, 9]
```

Egy könyvtár tartalmát egyszerűen beolvashatjuk a [*glob*](https://docs.python.org/3/library/glob.html) modul segítségével. Példa:

```python
import glob
txts = glob.glob('*.txt')
# ['a.txt', 'b.txt', 'c.txt']
```

Python-ból külső programot futtatni a [*subprocess*]() modul *run()* függvényével lehet. Példa:

```python
from subprocess import run, PIPE
res = run(['ls', '-'], stdout=PIPE)
# CompletedProcess ...
res = res.stdout
# b'a.txt\nb.txt\nc.txt\n'
```

In [ ]:
%%bash
# 1. feladat: html-ek letöltése

rm -rf train && mkdir train
wget -P train/ ... http://clara.nytud.hu/~kk120/
ls train/*.html

In [ ]:
%%bash
# 1.A. feladat: számozatlan html-ek letöltése

rm -rf train && mkdir train
wget -P train/ ... http://clara.nytud.hu/~kk120/
ls train/*.html

In [ ]:
# 2. feladat: paragrafusok kinyerése és kiírása

import glob
from bs4 import BeautifulSoup as BS

files = glob.glob('train/*.html')
print(files)

In [ ]:
# 2. feladat: folytatás...

soups = [... for file in files]

def get_paragraphs(soup):
    res = ...
    ...
    return res

[open(file+'.txt', 'w').write(get_paragraphs(soup)) for file, soup in soups]

!ls train/*.txt

In [ ]:
%%bash
# 3. feladat: pdf-ek letöltése

wget -P train/ ... http://clara.nytud.hu/~kk120/

ls train/*.pdf

In [ ]:
# 4. feladat: pdf-ek szöveggé alakítása

from subprocess import run, PIPE

files = glob.glob('train/*.pdf')[0:3]
texts = [run(...) for x in files[0:3]]
texts = [text.stdout for text in texts]

print(texts[2])

## Irodalom

crawling:

* [wget](https://www.gnu.org/software/wget/)
* [curl](https://curl.haxx.se/)
* [scrapy](https://scrapy.org/)

boilerplate removal, extraction:

* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [jusText](https://github.com/miso-belica/jusText), [demo](https://nlp.fi.muni.cz/projects/justext/)
* [pdftotext](https://en.wikipedia.org/wiki/Pdftotext)
* [tika](https://tika.apache.org/)